<a href="https://colab.research.google.com/github/tlt28544/tlt28544/blob/main/HW6_Leslie_Tan_TT2976.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

from google.colab import drive
drive.mount('/content/drive')
dirname = '/content/drive/MyDrive/DDMIF/HW6/'

Mounted at /content/drive


In [4]:
df1 = pd.read_csv(dirname+'Optimization_with_Transaction_Cost_weight.txt', delimiter = "\t", header = None)
df1.columns=['ticker','wb']

In [5]:
df2 = pd.read_csv(dirname+'Optimization_with_Transaction_Cost_return.txt', delimiter = "\t", header = None)
df2.columns = ['ticker'] + [str(i) for i in range(1,61)]

In [7]:
# Extract weights and returns
wb = df1['wb'].values
returns = df2.iloc[:, 1:].values

In [8]:
# Step 2: Calculate mean returns (annualized) and variance-covariance matrix (annualized)
mu = np.mean(returns, axis=1) * 12  # Annualized mean returns
cov_matrix = np.cov(returns) * 12   # Annualized variance-covariance matrix

In [9]:
# Define the risk aversion parameter
A = 2
N = 50  # Number of stocks

In [10]:
# Constraints for the first optimization problem
def constraints_1(w):
    return np.sum(w) - 1  # Sum of weights must be 1

In [11]:
# Bounds for weights (0 <= wa <= 1)
bounds = [(0, 1) for _ in range(N)]

# Objective function for the first optimization problem
def objective_1(wa):
    return - (wa @ mu - A * wa @ cov_matrix @ wa)

In [12]:
# Initial guess (uniform weights)
initial_guess = np.ones(N) / N

In [14]:
from scipy.optimize import minimize

In [15]:
# Step 3: Solve the first optimization problem
constraints = {'type': 'eq', 'fun': constraints_1}
result_1 = minimize(objective_1, initial_guess, bounds=bounds, constraints=constraints)


In [16]:
# Extract the optimal weights (w_hat)
w_hat = result_1.x


In [18]:
# Step 4: Define transaction costs vector c based on signs of (w_hat - wb)
c = np.where(w_hat >= wb, 0.001, -0.001)

# Constraints for the second optimization problem
def constraints_2(w):
    return np.sum(w) - 1  # Sum of weights must be 1


In [19]:
# Bounds based on the conditions given in the problem
bounds_2 = []
for i in range(N):
    if wb[i] <= w_hat[i]:
        bounds_2.append((wb[i], 1))
    else:
        bounds_2.append((0, wb[i]))

In [20]:
# Objective function for the second optimization problem
def objective_2(wa):
    transaction_cost = (wa - wb) @ c
    return - (wa @ mu - transaction_cost - A * wa @ cov_matrix @ wa)


In [21]:
# Step 5: Solve the second optimization problem
result_2 = minimize(objective_2, initial_guess, bounds=bounds_2, constraints=constraints)

# Extract the final optimal weights (wa)
wa_optimal = result_2.x

In [22]:
# Prepare results for output
results = pd.DataFrame({
    'Ticker': df1['ticker'],
    'Weight Before (wb)': wb,
    'Optimal Weight (w_hat)': w_hat,
    'Final Optimal Weight (wa)': wa_optimal
})


In [25]:
results

,Ticker,Weight Before (wb),Optimal Weight (w_hat),Final Optimal Weight (wa)
0,ABC,0.000000,0.000000e+00,2.434589e-09
1,ANZBY,0.000000,0.000000e+00,3.571281e-09
2,APD,0.000000,0.000000e+00,3.198071e-09
3,APU,0.000000,0.000000e+00,3.043301e-09
4,ASH,0.000000,5.157523e-10,4.456741e-10
5,ATO,0.000000,0.000000e+00,4.515289e-10
6,BDX,0.050704,5.782177e-02,5.781539e-02
7,BEN,0.000000,7.138868e-13,9.857403e-10
8,BJS,0.000000,5.385132e-14,2.156983e-09
9,CFFN,0.177447,1.400100e-01,1.400257e-01
